## Escenario 1: Un cientifico de datos participando de una competición en kaggle

MLflow setup:
* Tracking server: no
* Backend store: sistema de archivos local
* Artifacts store: sistema de archivos local

Usando MLFlow Ui podemos revisar los resultados

In [1]:
import mlflow

En este caso estamos usando el path actual para definir donde queremos guardar las ejecuciones

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns'


Define la ubicación donde se almacenarán los metadatos de los experimentos. En este caso, apunta al directorio mlruns dentro de la carpeta scenarios.

In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns/552618715103696137', creation_time=1755745784606, experiment_id='552618715103696137', last_update_time=1755745784606, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns/0', creation_time=1755745751751, experiment_id='0', last_update_time=1755745751751, lifecycle_stage='active', name='Default', tags={}>]

### Creemos un experimento y logguemos la corrida

Hagamos una prueba con un dataset de juguete, aca definimos siempre un experimento de mlflow de la siguiente manera:

#### Definición del Experimento

```python 
mlflow.set_experiment("my-experiment-1")
```

Crea o selecciona un experimento llamado "my-experiment-1" donde se registrarán todas las ejecuciones relacionadas.

#### Ejecución del Experimento

```python
with mlflow.start_run():                    
    # Código del experimento
```

El contexto with mlflow.start_run() inicia automáticamente una nueva ejecución y la finaliza al salir del bloque.


## Flujo del Experimento

1. Carga de Datos

- Utiliza el dataset de iris de scikit-learn como ejemplo.

2. Definición de Parámetros: 
- C: Parámetro de regularización para LogisticRegression
- random_state: Semilla para reproducibilidad

- mlflow.log_params() registra estos parámetros para futura referencia

3. Entrenamiento del Modelo 

- Entrena un modelo de regresión logística con los parámetros especificados.

4. Evaluación y Logging de Métricas

- Calcula predicciones en el conjunto de entrenamiento
- Registra la métrica de precisión (accuracy) en MLflow

5. Logging del Modelo

- Guarda el modelo entrenado como un artifact, permitiendo su reutilización posterior.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, f1_score, precision_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.3, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [14]:
# import Random Forest

from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run():
    # Código del experimento
    params = {"n_estimators": 100, "random_state": 42}

    X, y = load_iris(return_X_y=True)

    mlflow.log_params(params)

    rf = RandomForestClassifier(**params).fit(X, y)
    y_pred = rf.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    

    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")
    

default artifacts URI: 'file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns/552618715103696137/af8b8bac68624244942c719e2bf9e05c/artifacts'


## Exploración de Resultados

### Búsqueda de Experimentos

Lista todos los experimentos disponibles en el tracking server.


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns/552618715103696137', creation_time=1755745784606, experiment_id='552618715103696137', last_update_time=1755745784606, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns/0', creation_time=1755745751751, experiment_id='0', last_update_time=1755745751751, lifecycle_stage='active', name='Default', tags={}>]

Filtrando por texto: 

In [8]:
mlflow.search_experiments(filter_string="name = 'my-experiment-1'")

[<Experiment: artifact_location='file:///Users/macano/Desktop/code_personal/MLOps_UdM/02-Experiment-Tracking/scenarios/mlruns/552618715103696137', creation_time=1755745784606, experiment_id='552618715103696137', last_update_time=1755745784606, lifecycle_stage='active', name='my-experiment-1', tags={}>]

Filtrando por ID

In [15]:
mlflow.search_runs(experiment_ids=['552618715103696137'])

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.random_state,params.n_estimators,params.C,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.user
0,af8b8bac68624244942c719e2bf9e05c,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:34:14.835000+00:00,2025-09-16 01:34:14.887000+00:00,1.000000,42,100,None,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,bittersweet-fly-391,macano
1,63381b445b794d63b82a04b658a4e22d,552618715103696137,FAILED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:28:54.644000+00:00,2025-09-16 01:28:54.658000+00:00,0.966667,42,None,0.3,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,sneaky-yak-413,macano
2,de619d11268d403093e882b657a246b4,552618715103696137,FAILED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:28:33.827000+00:00,2025-09-16 01:28:33.842000+00:00,0.966667,42,None,0.3,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,carefree-roo-713,macano
3,77018ef1003f4cdb87c9c32af35cce6b,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:26:44.120000+00:00,2025-09-16 01:26:45.221000+00:00,0.966667,42,None,0.2,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,unruly-midge-449,macano
4,b0169c5f69fe41dca1e80de1ffe405eb,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:24:54.826000+00:00,2025-09-16 01:24:56.254000+00:00,0.960000,42,None,0.1,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,unruly-crane-956,macano
5,0e7b2e5697784d5a82768712a0f5a49d,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-08-21 03:21:04.783000+00:00,2025-08-21 03:21:06.071000+00:00,0.960000,42,None,0.1,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,selective-crab-621,macano
6,af10e7b4e907433991f1927dc2ba0e5e,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-08-21 03:09:44.680000+00:00,2025-08-21 03:09:46.166000+00:00,0.960000,42,None,0.1,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,defiant-fly-919,macano


In [16]:
mlflow.search_runs(experiment_ids=['552618715103696137']).sort_values(by="metrics.accuracy", ascending=False)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.random_state,params.n_estimators,params.C,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.user
0,af8b8bac68624244942c719e2bf9e05c,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:34:14.835000+00:00,2025-09-16 01:34:14.887000+00:00,1.000000,42,100,None,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,bittersweet-fly-391,macano
1,63381b445b794d63b82a04b658a4e22d,552618715103696137,FAILED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:28:54.644000+00:00,2025-09-16 01:28:54.658000+00:00,0.966667,42,None,0.3,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,sneaky-yak-413,macano
2,de619d11268d403093e882b657a246b4,552618715103696137,FAILED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:28:33.827000+00:00,2025-09-16 01:28:33.842000+00:00,0.966667,42,None,0.3,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,carefree-roo-713,macano
3,77018ef1003f4cdb87c9c32af35cce6b,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:26:44.120000+00:00,2025-09-16 01:26:45.221000+00:00,0.966667,42,None,0.2,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,unruly-midge-449,macano
4,b0169c5f69fe41dca1e80de1ffe405eb,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-09-16 01:24:54.826000+00:00,2025-09-16 01:24:56.254000+00:00,0.960000,42,None,0.1,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,unruly-crane-956,macano
5,0e7b2e5697784d5a82768712a0f5a49d,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-08-21 03:21:04.783000+00:00,2025-08-21 03:21:06.071000+00:00,0.960000,42,None,0.1,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,selective-crab-621,macano
6,af10e7b4e907433991f1927dc2ba0e5e,552618715103696137,FINISHED,file:///Users/macano/Desktop/code_personal/MLO...,2025-08-21 03:09:44.680000+00:00,2025-08-21 03:09:46.166000+00:00,0.960000,42,None,0.1,LOCAL,/Users/macano/Desktop/code_personal/MLOps_UdM/...,defiant-fly-919,macano


### Interacción con el model registry

Permite interactuar programáticamente con el tracking server y model registry.

In [12]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [13]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")